<a href="https://colab.research.google.com/github/ADI201998/sparse-to-depth-GAN/blob/master/depth_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install tensorflow-gpu

     |████████████████████████████████| 377.0MB 80kB/s 


In [0]:
!pip install -U -q PyDrive

In [0]:
from google.colab.patches import cv2_imshow
import cv2
from google.colab import files
from matplotlib import pyplot as plt
import os
import h5py
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv2D, UpSampling2D, Conv2DTranspose
from keras.layers import ZeroPadding2D, Activation, BatchNormalization, MaxPooling2D, Dense, Add, Flatten, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Input, Dropout
from keras import losses
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

input_shape_generator = (480, 640, 4)
input_shape_discriminator = (480, 640, 1)

Using TensorFlow backend.


In [0]:
def dense_to_sparse(rgb, depth):
        mask_keep = depth > 0
        if np.inf is not np.inf:
                mask_keep = np.bitwise_and(mask_keep, depth <= np.inf)
        n_keep = np.count_nonzero(mask_keep)
        if n_keep == 0:
                return mask_keep
        else:
                prob = float(200) / n_keep
                return np.bitwise_and(mask_keep, np.random.uniform(0, 1, depth.shape) < prob)

In [0]:
            
def create_sparse_depth(rgb, depth):
        mask_keep = dense_to_sparse(rgb, depth)
        sparse_depth = np.zeros(depth.shape)
        sparse_depth[mask_keep] = depth[mask_keep]
        return sparse_depth

In [0]:
def create_rgbd(rgb, depth):
        sparse_depth = create_sparse_depth(rgb, depth)
        rgbd = np.append(rgb, np.expand_dims(sparse_depth, axis=2), axis=2)
        rgbd = rgbd[np.newaxis, :, :, :]
        return rgbd

In [0]:
def dataset():
        X_train = np.empty((280*6, 480, 640, 4))
        Y_train = np.empty((280*6, 480, 640, 1))
        rootdir_train = '/content/drive/My Drive/Dataset/train/'
        rootdir_val = '/content/drive/My Drive/Dataset/val/official/'
        i = 0
        for subdir, dirs, files in sorted(os.walk(rootdir_train)):
                files = sorted(files)
                if len(files) != 0:
                    for k in range(6):
                        path = subdir + '/' + files[k]
                        h5f = h5py.File(path, "r")
                        rgb = np.array(h5f['rgb'])
                        rgb = np.transpose(rgb, (1, 2, 0))
                        depth = np.array(h5f['depth'])
                        depth  = depth /9
                        X_train[i] = create_rgbd(rgb, depth)
                        Y_train[i] = depth[:, :, np.newaxis]
                    print(i)
                    i = i+1
        return X_train, Y_train

In [0]:
def generator_model():
    #encoder
    X_input = Input(input_shape_generator)
    X = Conv2D(64, 3, data_format = 'channels_last', padding='same')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(128, 3, data_format = 'channels_last', padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(256, 3,data_format = 'channels_last', padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(256, 1, data_format = 'channels_last', padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X1 = X
    X = Conv2D(512, 3, data_format = 'channels_last', padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(512, 1, data_format = 'channels_last', padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X2 = X
    X = Conv2D(512, 3, data_format = 'channels_last', padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2,2))(X) 
    X3 = X
    X = Conv2D(512, 3, data_format = 'channels_last', padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((1,2))(X)
    X = BatchNormalization(axis=3)(X)
    X = Dropout(0.4)(X)

    #decoder
    X = Conv2DTranspose(512, 3, data_format = 'channels_last', padding='same')(UpSampling2D((1,2))(X))
    X = BatchNormalization(axis=3)(X)
    X = Concatenate(axis = 3)([X, X3])
    X = Activation('relu')(X)
    
    #X = Dropout(0.3)(X)

    X = Conv2DTranspose(512, 3, data_format = 'channels_last', padding='same')(UpSampling2D((2,2))(X))
    X = BatchNormalization(axis=3)(X)
    X = Concatenate(axis = 3)([X, X2])
    X = Activation('relu')(X)
    
    #X = Dropout(0.3)(X)

    X = Conv2DTranspose(256, 3, data_format = 'channels_last', padding='same')(UpSampling2D((2,2))(X))
    X = BatchNormalization(axis=3)(X)
    X = Concatenate(axis = 3)([X, X1])
    X = Activation('relu')(X)
   

    X = Conv2DTranspose(128, 3, data_format = 'channels_last', padding='same')(UpSampling2D((2,2))(X))
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = Conv2DTranspose(64, 3, data_format = 'channels_last', padding='same')(UpSampling2D((2,2))(X))
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = Conv2DTranspose(1, 3, data_format = 'channels_last', padding='same')(UpSampling2D((2,2))(X))
    X = BatchNormalization(axis=3)(X)
    X_out = Activation('tanh')(X)

    model = Model(input = X_input, output = X_out, name='Generator')
    return model

In [0]:
 def discriminator_model():
    X_input = Input(input_shape_discriminator)
    X = Conv2D(64, kernel_size=(3,3), data_format = 'channels_last', padding='same')(X_input)
    X = Activation('relu')(X)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(128, kernel_size=(3,3), data_format = 'channels_last', padding='same')(X)
    X = Activation('relu')(X)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(256, kernel_size=(3,3), data_format = 'channels_last', padding='same')(X)
    X = Activation('relu')(X)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(256, kernel_size=(3,3), data_format = 'channels_last', padding='same')(X)
    X = Activation('relu')(X)
    X = BatchNormalization(axis=3)(X)
    X = Dropout(0.2)(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(512, kernel_size=(3,3), data_format = 'channels_last', padding='same')(X)
    X = Activation('relu')(X)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(512, kernel_size=(3,3), data_format = 'channels_last', padding='same')(X)
    X = Activation('relu')(X)
    X = BatchNormalization(axis=3)(X)
    X = Dropout(0.3)(X)
    X = MaxPooling2D((2,2))(X)
    X = Flatten()(X)
    X = Dropout(0.5)(X)
    X_out = Dense(1, activation = 'sigmoid')(X)

    model = Model(input =  X_input, output = X_out, name='Discriminator')
    return model

In [0]:
def gan_model(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(input_shape_generator)
    x = generator(gan_input)
    gan_output = discriminator(x)

    gan = Model(input = gan_input, output = [x, gan_output], name='GAN')
    return gan

In [0]:
batch_size = 16
epochs = 110
input_shape = (480, 640, 4)

## INITIALIZE MODELS  ##
print("----------------------INITIALIZING MODELS-----------------------")
generator = generator_model()       
#discriminator = discriminator_model()
#gan = gan_model(generator, discriminator)

## COMPILE MODELS ##
print("----------------------COMPILING MODELS-----------------------")
#discriminator.compile(optimizer = Adam(1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
#gan.compile(optimizer = Adam(1e-4), loss = ['mae', 'binary_crossentropy'], loss_weights=[0.999, 0.001])
generator.compile(optimizer = Adam(8e-3), loss = 'mae')
i = 0

##  INPUT  ##
print("----------------------GETTING INPUTS-----------------------")
X_train, Y_train = dataset()
X_train = (X_train - 127.5)/127.5
Y_train = (Y_train - 127.5)/127.5
#print(X_train.shape, Y_train.shape)

valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

#gen = load_model('/content/generator_model.h5')

print("----------------------STARTING TRAINING-----------------------")

#print(discriminator.metrics_names, gan.metrics_names)
print(generator.metrics_names)
'''''''''
for epoch in range(epochs):

    # SELECTING RANDOM BATCH OF IMAGES                      
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]
    op_imgs = Y_train[idx]

    # GENERATE NEW IMAGES
    generated_img = generator.predict(imgs)

    discriminator.trainable = True

    # TRAIN DISCRIMINATOR
    d_loss_real = discriminator.train_on_batch(op_imgs, valid)
    d_loss_fake = discriminator.train_on_batch(generated_img, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    discriminator.trainable = False
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]
    op_imgs = Y_train[idx]
    # TRAIN GENERATOR
    g_loss = gan.train_on_batch(imgs, [op_imgs, valid])                           

    print("Epochs : ",epoch," D_loss : ", d_loss, " G_loss : ", g_loss)
    

print("----------------------TRAINING ENDS-----------------------")
generator.save('generator_model.h5')
discriminator.save('disriminator_model.h5')
'''''''''
generator.fit(x = X_train, y = Y_train, batch_size = 8, epochs = 10, shuffle=True, validation_split=0.2)
generator.save('generator_model.h5')
json_string = generator.to_json()
open('generator.json', 'w').write(json_string)
generator.save_weights('weights.h5')
print("DOWNLOADING.....")
model_file1 = drive.CreateFile({'title' : 'generator_model.h5'})
model_file2 = drive.CreateFile({'title' : 'generator.json'})
model_file3 = drive.CreateFile({'title' : 'weights.h5'})
model_file1.SetContentFile('generator_model.h5')
model_file2.SetContentFile('weights.h5')
model_file3.SetContentFile('generator.json')
model_file1.Upload()
model_file2.Upload()
model_file3.Upload()
# download to google drive
drive.CreateFile({'id': model_file1.get('id')})
drive.CreateFile({'id': model_file2.get('id')})
drive.CreateFile({'id': model_file3.get('id')})
#files.download('generator.json')
#files.download('weights.h5')
print("DONE.....")


----------------------INITIALIZING MODELS-----------------------







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

----------------------COMPILING MODELS-----------------------



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="Generator", inputs=Tensor("in..., outputs=Tensor("ac...)`


----------------------GETTING INPUTS-----------------------
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
26

In [0]:
X_test = np.empty((168, 480, 640, 4))
Y_test = np.empty((168, 480, 640, 1))
rootdir_train = '/content/drive/My Drive/Dataset/train/'
rootdir_val = '/content/drive/My Drive/Dataset/val/official/'
i = 0
for subdir, dirs, files in sorted(os.walk(rootdir_val)):
        files = sorted(files)
        for k in range(168):
            path = subdir + '/' + files[k]
            h5f = h5py.File(path, "r")
            rgb = np.array(h5f['rgb'])
            rgb = np.transpose(rgb, (1, 2, 0))
            depth = np.array(h5f['depth'])
            depth  = depth /9
            X_test[i] = create_rgbd(rgb, depth)
            Y_test[i] = depth[:, :, np.newaxis]


In [0]:
valid = np.ones((8, 1))
fake = np.zeros((168, 1))
k = 0
for i in range(1,22):
        imgs = generator.predict(X_test[k:8*i, :, :, :])
        l1 = generator.evaluate(x = (X_test[k:8*i, :, :, :]-127.5)/127.5, y = (Y_test[k:8*i, :, :, :]-127.5)/127.5)
        print("L1 : ",l1)
        k = k+8

8/8 [==============================] - 1s 89ms/step
L1 :  0.0016242156270891428
8/8 [==============================] - 1s 86ms/step
L1 :  0.000815560226328671
8/8 [==============================] - 1s 72ms/step
L1 :  0.0008155559189617634
8/8 [==============================] - 1s 71ms/step
L1 :  0.000815560226328671
8/8 [==============================] - 1s 72ms/step
L1 :  0.000815560226328671
8/8 [==============================] - 1s 71ms/step
L1 :  0.000815560226328671
8/8 [==============================] - 1s 73ms/step
L1 :  0.000815560226328671
8/8 [==============================] - 1s 72ms/step
L1 :  0.000815560226328671
8/8 [==============================] - 1s 73ms/step
L1 :  0.000815560226328671
8/8 [==============================] - 1s 71ms/step
L1 :  0.000815560226328671
8/8 [==============================] - 1s 72ms/step
L1 :  0.000815560226328671
8/8 [==============================] - 1s 72ms/step
L1 :  0.000815560226328671
8/8 [==============================] - 1s 72ms/ste